In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tabulate import tabulate
import pickle

from ga_utils import initialize_base_variables, models

In [4]:
X_bow, y_score, logmodel = initialize_base_variables("../dataset/amazon.csv")

In [12]:
ga_path = '../pickles/ga/30runs/'
kbga_path = '../pickles/kbga/30runs/'
kb_path = "../pickles/kb/"
de_path = '../pickles/de/'
kbde_path = '../pickles/kbde/'


ga_files = ['30_run_az_ga_ts_pt.pkl',
            '30_run_imdb_ga_ts_pt.pkl',
            '30_run_yelp_ga_ts_pt.pkl',
            '30_run_az_ga_ts.pkl',
            '30_run_imdb_ga_ts.pkl',
            '30_run_yelp_ga_ts.pkl'
            ]

kb_files = ['30_run_az_kbip_ts_pt.pkl',
            '30_run_az_kbps_ts_pt.pkl',
            '30_run_az_kbco_ts_pt.pkl',
            '30_run_az_kbmu_ts_pt.pkl',
            ]

kbga_files = [
            '30_run_az_kbga_vp_ts.pkl',
            '30_run_az_kbga_es50_100_ts.pkl',
            '30_run_imdb_kbga_vp_ts.pkl',
            '30_run_imdb_kbga_es50_100_ts.pkl',
            '30_run_yelp_kbga_vp_ts.pkl',
            '30_run_yelp_kbga_es50_100_ts.pkl',
            ]

de_files = [
            '30_run_az_de.pkl',
            'n_run_az_de_crt.pkl',
            'n_run_az_de_crt1.pkl',
            ]

kbde_files = [
            '30_run_az_kbde.pkl',
            'n_run_az_kbde_crt.pkl',
            'n_run_az_kbde_crt1.pkl',
            ]

In [16]:
rf = open(ga_path+ga_files[0], 'rb')
ga_az = pickle.load(rf)

rf = open(kbga_path+kbga_files[0], 'rb')
kbga_az = pickle.load(rf)

# Collect Common indexes from res of single run evolution

In [ ]:
def predicted_sentiment_ratio(y_test, predictions):
    positive_percent = np.count_nonzero(predictions==1)*100//len(predictions)
    negative_percent = np.count_nonzero(predictions==0)*100//len(predictions)
    # print(f"Y_test: pos/neg percentage ~ {np.count_nonzero(y_test==1)*100//len(y_test)}, {np.count_nonzero(y_test==0)*100//len(y_test)}")
    return positive_percent, negative_percent

In [19]:
chromo_len_in_each_gen = []
for i in chromo_set_1:
    chromo_len_in_each_gen.append(np.count_nonzero(i==1))
chromo_len_in_each_gen = np.array(chromo_len_in_each_gen)
chromo_len_in_each_gen

array([100, 100, 338, 480, 480, 480, 480, 480, 480, 375, 375, 666, 666,
       666, 666, 666, 666, 666, 534, 534, 534, 534, 576, 709, 709, 709,
       709, 709, 709, 709])

In [21]:
common_index = dict()
for chromo in chromo_set_1:
    for i in range(len(chromo)):
        if chromo[i] == 1:
            common_index[i] = common_index.setdefault(i, 0) + 1

sorted_common_index = np.array(sorted(common_index.items(), key=lambda x: x[1], reverse=True)[:200])
sorted_common_index[:, 0]

array([ 156,  594,  779,   81,  179,  220,  272,  723, 1190, 1343, 1389,
       1432,   13,  858,   79,  458,  517,  603, 1057, 1070, 1195, 1470,
        744,  869, 1376,  732,  902, 1100,  151,  167,  170,  422,  979,
       1182, 1248,  866, 1064, 1305, 1319,  564,  535,  565,  623,  929,
        149,  251,  261, 1033, 1465,  176,  207,  338,  361,  407,  419,
        432,  495,  515,  525,  532,  590,  713,  725,  750,  754, 1165,
       1180, 1246, 1321, 1375, 1510, 1157, 1467,  104,  113,  187,  288,
        523,  897, 1292, 1496,  938, 1000, 1317,  304,  519,  599,  656,
        693,  896,  915,  997, 1052, 1264,  101,  139,  198,  218,  253,
        279,  283,  313,  352,  790,  967, 1115, 1130, 1132, 1172, 1213,
       1255, 1294, 1330, 1335, 1354, 1380,   23,  102,  105,  490,  544,
        583,  619,  734, 1454,  126,  178,  186,  188,  355,  357,  425,
        868, 1023, 1096, 1120, 1129, 1136, 1137, 1140, 1204, 1262, 1309,
       1348, 1360, 1502,   33,   46,   65,  426,  4

In [ ]:
def test_accuracy(selected_genes):
    st = time.time()
    logmodel.fit(X_train[:, selected_genes], Y_train)
    et = time.time()

    predictions = logmodel.predict(X_test[:, selected_genes])
    ratio = predicted_sentiment_ratio(Y_test, predictions)
    return accuracy_score(Y_test, predictions), et-st, ratio

In [ ]:
def get_intersection_genes(result_set):
    size_of_chromo = None
    common_indexes = {}
    best_chromo_in_each_run = []
    for res in result_set:
        best_chromo = res[0][-1]   
        size_of_chromo = len(best_chromo)
        chromo_features = np.where(best_chromo != 0)[0]
        best_chromo_in_each_run.append([best_chromo, chromo_features.shape[0], res[1][-1]])

        for index in chromo_features:
            common_indexes[index] = common_indexes.setdefault(index, 0) + 1
            
    sorted_common_indexes = np.array(sorted(common_indexes.items(), key=lambda x: x[1], reverse=True))
    
    return sorted_common_indexes, best_chromo_in_each_run

In [ ]:
def tabulate_results(chromo_set):
    print('Actual chromosome length in X_Test: ', X_bow.shape[1])
    print('\nRandomForest with full length -', all_models_score_table.iloc[0, 1:], "\n")
    # print('Total Common genes count ~(-1): ', top_i_common_indexes)

    accuracy = []
    features_count = []
    time_to_train = []
    sentiment_ratio = []

    sorted_common_indexes, best_chromo_in_each_run = get_intersection_genes(chromo_set)
    common_indexes_count = sorted_common_indexes[:, 1]
    # print("Total common indexes:", sorted_common_indexes.shape[0])

    count_thresholds = list(set(common_indexes_count))[::-1]
    print("Intersection frequency:", count_thresholds, "\n")

    for i in count_thresholds:
        selected_features  = sorted_common_indexes[np.where(sorted_common_indexes[:, 1] >= i), 0][0]
        features_count.append(len(selected_features))
        acc, ttt, sr = test_accuracy(selected_features)
        accuracy.append(acc)
        time_to_train.append(ttt)
        sentiment_ratio.append(sr)

    
    table_data = {'Count Threshold': count_thresholds,
                'Feature count': features_count,
                'Accuracy': accuracy,
                'Time Taken to Train': time_to_train,
                'Sentiment Ratio(p/n)': sentiment_ratio
                }
    print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))
    return table_data, best_chromo_in_each_run
